In [1]:
import pandas as pd
from PIL import Image
pix = 32
traindf = pd.read_csv("./train.csv")
"""
Resizing images to 32 x 32 pixels

for index,row in traindf.iterrows():
    img = Image.open("../Train/"+row[0])
    img = img.resize((pix,pix), Image.ANTIALIAS)
    img.save("../Train/"+row[0]) 
"""

'\nfor index,row in traindf.iterrows():\n    img = Image.open("../Train/"+row[0])\n    img = img.resize((pix,pix), Image.ANTIALIAS)\n    img.save("../Train/"+row[0]) \n'

In [2]:
testdf = pd.read_csv("./test.csv")
"""
Resizing images to 32 x 32 pixels

for index , row in testdf.iterrows():
    img = Image.open("../Test/"+row[0])
    img = img.resize((pix,pix), Image.ANTIALIAS)
    img.save("../Test/"+row[0]) 
"""

'\nfor index , row in testdf.iterrows():\n    img = Image.open("../Test/"+row[0])\n    img = img.resize((pix,pix), Image.ANTIALIAS)\n    img.save("../Test/"+row[0]) \n'

In [3]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
from PIL import Image
import os
agedict = {'YOUNG':1 , 'MIDDLE' :2 , 'OLD':3}
class IMFDBDataset(Dataset):
    """IMFDB dataset"""

    def __init__(self, csv_file, root_dir, transform=None):
        self.actor_image = pd.read_csv(csv_file)
        self.imgname = self.actor_image.iloc[:,0]
        self.imgage = self.actor_image.iloc[:,1]
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.actor_image.index)

    def __getitem__(self, idx):
        img_name = self.root_dir + "/" + self.imgname[idx]
        image = Image.open(img_name)
        age = agedict[self.imgage[idx]]
        if self.transform:
            image = self.transform(image)
        return image,age

In [47]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
fullset = IMFDBDataset(csv_file='./train.csv',root_dir='../Train',transform = transform)
trainset, testset = torch.utils.data.random_split(fullset, [14000, 5906])

In [48]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=20,shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=20,shuffle=True, num_workers=2)

In [49]:
import numpy as np
"""
Plotting Random train images

import matplotlib.pyplot as plt
from torchvision import transforms, utils
import numpy as np
def imshow(img):
    img = img/2 + 0.5     # unnormalize
    npimg = img.numpy()
    x = np.transpose(npimg, (1, 2, 0))
    plt.imshow(x)

dataiter = iter(trainloader)
images ,label = dataiter.next()
imshow(utils.make_grid(images))
"""

'\nPlotting Random train images\n\nimport matplotlib.pyplot as plt\nfrom torchvision import transforms, utils\nimport numpy as np\ndef imshow(img):\n    img = img/2 + 0.5     # unnormalize\n    npimg = img.numpy()\n    x = np.transpose(npimg, (1, 2, 0))\n    plt.imshow(x)\n\ndataiter = iter(trainloader)\nimages ,label = dataiter.next()\nimshow(utils.make_grid(images))\n'

In [50]:
import torch.nn as nn
import torch.nn.functional as F
class AGE(nn.Module):
    def __init__(self):
        super(AGE, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 3,1,(1,1))
        self.conv = nn.Conv2d(10, 30, 3,1,(1,1))
        self.batch_0=nn.BatchNorm2d(30)
        self.conv2 = nn.Conv2d(30, 60, 3)
        self.batch_1=nn.BatchNorm2d(60)
        self.fc1 = nn.Linear(7*7*60, 500)
        self.batch_2=nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 300)
        self.batch_3=nn.BatchNorm1d(300)
        self.fc3 = nn.Linear(300, 100)
        self.fc4 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.batch_0(self.conv(x)))
        x = F.relu(self.batch_1(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 7*7*60)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
    def name(self):
        return "AGE"

import torch.optim as optim
model = AGE()
model.cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()

for epoch in range(45):
    # trainning
    ave_loss = 0
    for i,data in enumerate(trainloader,0):
        inputs, label = data
        inputs, label = inputs.cuda(), label.cuda()
        optimizer.zero_grad()
        out = model(inputs)
        loss = criterion(out,label)
        loss.backward()
        optimizer.step()
        ave_loss+= loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, ave_loss / 200))
            ave_loss = 0.0
print("Finished Training")



[1,   200] loss: 1.261
[1,   400] loss: 0.930
[1,   600] loss: 0.898
[2,   200] loss: 0.807
[2,   400] loss: 0.782
[2,   600] loss: 0.776
[3,   200] loss: 0.729
[3,   400] loss: 0.737
[3,   600] loss: 0.737
[4,   200] loss: 0.707
[4,   400] loss: 0.689
[4,   600] loss: 0.687
[5,   200] loss: 0.665
[5,   400] loss: 0.665
[5,   600] loss: 0.657
[6,   200] loss: 0.619
[6,   400] loss: 0.640
[6,   600] loss: 0.624
[7,   200] loss: 0.599
[7,   400] loss: 0.610
[7,   600] loss: 0.596
[8,   200] loss: 0.573
[8,   400] loss: 0.561
[8,   600] loss: 0.579
[9,   200] loss: 0.545
[9,   400] loss: 0.538
[9,   600] loss: 0.534
[10,   200] loss: 0.513
[10,   400] loss: 0.491
[10,   600] loss: 0.512
[11,   200] loss: 0.452
[11,   400] loss: 0.483
[11,   600] loss: 0.466
[12,   200] loss: 0.420
[12,   400] loss: 0.433
[12,   600] loss: 0.438
[13,   200] loss: 0.391
[13,   400] loss: 0.391
[13,   600] loss: 0.416
[14,   200] loss: 0.327
[14,   400] loss: 0.366
[14,   600] loss: 0.348
[15,   200] loss: 0

In [51]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(total)
print(correct)
print(100 * correct / total)

5906
4709
79.73247544869623
